In [6]:
import os,io
from threading import Thread
from queue import Queue
import urllib.parse
from PIL import Image
import PySimpleGUI as sg
from selenium import webdriver

SCRIPT_DIR=os.path.dirname("__file__")
ui_events=Queue()

def show_widnow():
    layout=[
        [sg.Text('検索キーワードを入力してください')],
        [sg.InputText(key='-keyword-')],
        [sg.Button('検索'),sg.Button('終了')],
        [sg.Image(key="-image-")]
    ]
    window=sg.Window('画像検索',layout)

    while True:
        event,values=window.read(timeout=100,timeout_key="-timeout-")
        if event in [sg.WIN_CLOSED,'終了']:
            break
        if event=='検索':
            keyword=values['-keyword-']
            Thread(target=search_image,args=(keyword,)).start()
            continue
        if event=="-timeout-":
            if ui_events.empty():
                continue
            event,values=ui_events.get()
            if event=='update_image':
                window['-image-'].update(
                    data=load_image(values["path"]),
                    size=(500,400))
    window.close()

def search_image(keyword):
    key_enc=urllib.parse.quote(keyword)
    driver=webdriver.Chrome()
    driver.set_window_size(1024,800)
    url=f"https://google.com/search?q={key_enc}&sclient=img&tbm=isch"
    driver.get(url)
    driver.implicitly_wait(10)

    savefile=os.path.join(SCRIPT_DIR,f"shot-{key_enc}.png")
    driver.save_screenshot(savefile)
    driver.quit()
    ui_events.put(('update_image',{"path":savefile}))

def load_image(path):
    im=Image.open(path)
    im.thumbnail((500,400))
    bio=io.BytesIO()
    im.save(bio,format="PNG")
    return bio.getvalue()

if __name__=="__main__":
    show_widnow()